In [23]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm
import json
import concurrent.futures
import multiprocessing

MAX_THREADS = 30

In [134]:
title_df = pd.read_csv('zippedData/title.basics.tsv.gz', delimiter='\t', error_bad_lines=False)

/Users/michaelroth/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [135]:
title_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [136]:
title_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950519 entries, 0 to 8950518
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 614.6+ MB


In [137]:
title_df = title_df.query('titleType == "movie"')

In [138]:
title_df.startYear = title_df.startYear.apply(lambda x: np.nan if x == '\\N' else int(x))

In [139]:
title_df.dropna(inplace=True)

In [140]:
title_df.reset_index(drop=True, inplace=True)
title_df = title_df.query('isAdult == 0')
title_df.drop(labels=['endYear', 'isAdult'], axis=1, inplace=True)
title_df.head()

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt0000502,movie,Bohemios,Bohemios,1905.0,100,\N
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,1906.0,70,"Action,Adventure,Biography"
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,1907.0,90,Drama
3,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,1907.0,\N,Drama
4,tt0000630,movie,Hamlet,Amleto,1908.0,\N,Drama


In [141]:
title_df = title_df.query('startYear >= 1970 and startYear < 2022').sort_values('startYear').reset_index(drop=True)
title_df.head()

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt1445727,movie,Raman Ethanai Ramanadi,Raman Ethanai Ramanadi,1970.0,\N,\N
1,tt2178082,movie,Have You Ever Been on a Trip?,Have You Ever Been on a Trip?,1970.0,65,\N
2,tt4221520,movie,The Hornsey Film,The Hornsey Film,1970.0,60,Documentary
3,tt0350971,movie,Furra,Furra,1970.0,\N,Documentary
4,tt4202924,movie,"Myach, perchatka i kapitan","Myach, perchatka i kapitan",1970.0,90,"Comedy,Drama,Family"


In [142]:
full_details_df = title_df
full_details_df['budget'] = np.nan
full_details_df['domestic_box_office'] = np.nan
full_details_df['worldwide_box_office'] = np.nan
full_details_df['origin'] = np.nan
full_details_df['genres'] = np.nan
full_details_df['user_rating'] = np.nan
full_details_df['principals'] =  np.nan
full_details_df['director'] =  np.nan
full_details_df['rating'] =  np.nan
full_details_df['published_date'] =  np.nan
full_details_df['creator'] =  np.nan

In [143]:
def get_data_from_dataid(dataid, final_data_holder, soup, isNumber=False):
    data_li = soup.select(dataid)
    data = [y.get_text() for x in data_li for y in x.find_all(final_data_holder, {'class':"ipc-metadata-list-item__list-content-item"})]
    
    if len(data) > 0:
        if len(data) > 4:
            data = data[:4]
        if isNumber:
            data = int(''.join([x for x in data[0] if x.isnumeric()]))
        return data
    else:
        return

In [155]:
def download_url(index):  
    row = full_details_df.iloc[index]
    title_id = row.tconst
    url = 'http://imdb.com/title/' + title_id
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    metadata = soup.find('script')
    meta_json = json.loads(metadata.contents[0])
 
    meta_keys = meta_json.keys()
    
    try:
        row['budget'] = get_data_from_dataid('li[data-testid="title-boxoffice-budget"]', 'span', soup, isNumber = True)
        row['domestic_box_office'] = get_data_from_dataid('li[data-testid="title-boxoffice-grossdomestic"]', 'span', soup, isNumber = True)
        row['worldwide_box_office'] = get_data_from_dataid('li[data-testid="title-boxoffice-cumulativeworldwidegross"]', 'span', soup, isNumber = True)
        row['origin'] = get_data_from_dataid('li[data-testid="title-details-origin"]', 'a', soup)
        row['genres'] = meta_json['genre'] if 'genre' in meta_keys else np.nan
        row['user_rating'] = meta_json['aggregateRating']['ratingValue'] if 'aggregateRating' in meta_keys else np.nan
        row['principals'] = [{'name':x['name'], 'id':x['url'][-10:-1]} for x in meta_json['actor']] if 'actor' in meta_keys else np.nan
        row['director'] = [{'name':x['name'], 'id':x['url'][-10:-1]} for x in meta_json['director']] if 'director' in meta_keys else np.nan
        row['rating'] = meta_json['contentRating'] if 'contentRating' in meta_keys else np.nan
        row['published_date'] = meta_json['datePublished'] if 'datePublished' in meta_keys else np.nan
        row['creator'] = [{'id':x['url'][-10:-1]} for x in meta_json['creator']] if 'creator' in meta_keys else np.nan
    except:
        print('error')

    print(index, end='\r')
    full_details_df.iloc[index] = row
    
    

In [156]:
num_processes = multiprocessing.cpu_count()
num_processes

8

In [157]:
full_details_df['tconst'][:8].index

RangeIndex(start=0, stop=8, step=1)

In [159]:
pd.options.mode.chained_assignment = None

indices = list(full_details_df['tconst'].index)

threads = min(MAX_THREADS, len(indices))
    
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_url, indices)

KeyboardInterrupt: 

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,budget,domestic_box_office,worldwide_box_office,origin,user_rating,principals,director,rating,published_date,creator
0,tt1445727,movie,Raman Ethanai Ramanadi,Raman Ethanai Ramanadi,1970.0,\N,NaN,NaN,NaN,NaN,[India],5.8,"[{'name': 'M. Bhanumathi', 'id': 'nm8374130'},...","[{'name': 'P. Madhavan', 'id': 'nm1031045'}]",NaN,NaN,[{'id': 'nm9776645'}]
1,tt2178082,movie,Have You Ever Been on a Trip?,Have You Ever Been on a Trip?,1970.0,65,NaN,NaN,NaN,NaN,[United States],4.5,"[{'name': 'Jane Barber', 'id': 'nm4934580'}, {...",NaN,NaN,NaN,NaN
2,tt4221520,movie,The Hornsey Film,The Hornsey Film,1970.0,60,[Documentary],NaN,NaN,NaN,[United Kingdom],NaN,"[{'name': 'Charles Bagnall', 'id': 'nm5169541'...","[{'name': 'Patricia Holland', 'id': 'nm3041201'}]",NaN,NaN,NaN
3,tt0350971,movie,Furra,Furra,1970.0,\N,[Documentary],NaN,NaN,NaN,[Albania],NaN,NaN,NaN,NaN,NaN,[{'id': 'co0014426'}]
4,tt4202924,movie,"Myach, perchatka i kapitan","Myach, perchatka i kapitan",1970.0,90,"[Comedy, Drama, Family]",NaN,NaN,NaN,[Soviet Union],8.3,"[{'name': 'Vasili Chkhaidze', 'id': 'nm0158509...","[{'name': 'Temuri Palavandishvili', 'id': 'nm0...",NaN,1970-04-01,"[{'id': 'co0045536'}, {'id': 'co0114181'}, {'i..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,tt0385661,movie,The Harbormaster Returns,Dolaon hanggui sanai,1970.0,80,[Action],11000000.0,NaN,NaN,[South Korea],NaN,"[{'name': 'No-shik Park', 'id': 'nm1066229'}, ...","[{'name': 'Woo-yeol Jeon', 'id': 'nm1473420'}]",NaN,1970-06-05,"[{'id': 'co0059833'}, {'id': 'nm1481270'}]"
214,tt0202968,movie,Journey of Love,Kaze no bojo,1970.0,94,[Drama],NaN,NaN,NaN,[Japan],NaN,"[{'name': 'Sayuri Yoshinaga', 'id': 'nm0949045...","[{'name': 'Noboru Nakamura', 'id': 'nm0620205'}]",NaN,1971-04-02,[{'id': 'co0007001'}]
220,tt0070703,movie,Sensuous Sorceress,Skräcken har 1000 ögon,1970.0,99,[Horror],NaN,NaN,NaN,[Sweden],3.9,"[{'name': 'Hans Wahlgren', 'id': 'nm0906316'},...","[{'name': 'Torgny Wickman', 'id': 'nm0926967'}]",NaN,1970-09-28,"[{'id': 'co0012253'}, {'id': 'nm0412186'}, {'i..."
233,tt0145132,movie,The Notorious Cleopatra,The Notorious Cleopatra,1970.0,88,"[Comedy, Drama]",NaN,NaN,NaN,[United States],3.9,"[{'name': 'Loray White', 'id': 'nm0814488'}, {...","[{'name': 'Peter Perry Jr.', 'id': 'nm0186335'}]",NaN,1970-06-12,"[{'id': 'co0059485'}, {'id': 'nm0532585'}]"
